In [ ]:
#install libraries
!pip install pycox torchtuples

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.6/73.6 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.9/41.9 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.7/69.7 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 46.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.4/96.4 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.7/50.7 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.3/141.3 kB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 412.9/412.9 kB 24.4 MB/s eta 0:00:00
  Created wheel for feather-format: filename=feather_format-0.4.1-py3-none-any.whl size=2434 sha256=15a2f353c9ec68301847acc7424cef6c53e75f1791e0080d5604289e1ca4efd6
  Stored in directory: /root/.cache/pip/wheels/77/5b/0e/0e63d10b6353208a085a321ea2eed2578f220a77bb8a4bd7ab
Successfully built feather-format


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import torchtuples as tt
from pycox.models import DeepHitSingle
from pycox.evaluation import EvalSurv
from pycox.models import DeepHitSingle

pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Practicum Models/survival_episodes_cleaned.csv')
df

,randid,start_date,duration,event_observed,NOAAClimRegion,TempZone,HumidZone,bin_PlantSize_kW,plant_age_months,active_snow_tickets,snow_bin_ticket_minutes,snow_affected_assets,snow_production_level,snow_value_mm,total_daily_snow_mm,low_irradiation,cumulative_snow_mm,hurr_bin_ticket_minutes,hurr_affected_assets,hurr_production_level,HurricanePrep,HurricanePostInspection,hurricane,nearest_hurricane,wind_speed_mean,rain_value_mm,nearest_rain,storm_active_tickets,storm_bin_ticket_minutes,storm_affected_assets,storm_production_level,lightning,storm,duration_minutes_storm,nearest_storm,flood,duration_minutes_flood,nearest_flood,rain,weather_onset
0,C2S1,2018-10-13,1,1,West,T6,H4,large,25.0,No,NaN,NaN,Unknown,0.0,0.0,0,0.0,NaN,NaN,Unknown,No,No,0.0,354,2.677792,0.017805,0,No,NaN,NaN,Unknown,0,0.0,0,2420,0.0,0,2255,1.0,1
1,C2S1,2018-11-22,3,1,West,T6,H4,large,27.0,No,NaN,NaN,Unknown,0.0,0.0,0,0.0,NaN,NaN,Unknown,No,No,0.0,354,4.930521,1.136330,0,No,NaN,NaN,Unknown,0,0.0,0,2420,0.0,0,2255,1.0,1
2,C2S1,2018-11-29,3,1,West,T6,H4,large,27.0,No,NaN,NaN,Unknown,0.0,0.0,1,0.0,NaN,NaN,Unknown,No,No,0.0,354,5.715604,8.015464,0,No,NaN,NaN,Unknown,0,0.0,0,2420,0.0,0,2255,1.0,1
3,C2S1,2018-11-30,2,1,West,T6,H4,large,27.0,No,NaN,NaN,Unknown,0.0,0.0,1,0.0,NaN,NaN,Unknown,No,No,0.0,354,7.684687,0.239059,0,No,NaN,NaN,Unknown,0,0.0,0,2420,0.0,0,2255,1.0,1
4,C2S1,2018-12-05,2,1,West,T6,H4,large,27.0,No,NaN,NaN,Unknown,0.0,0.0,1,0.0,NaN,NaN,Unknown,No,No,0.0,354,1.672604,0.396306,0,No,NaN,NaN,Unknown,0,0.0,0,2420,0.0,0,2255,1.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7266,P2S1,2019-03-08,3,1,West,T1,H2,large,2.0,No,NaN,NaN,Unknown,0.0,0.0,0,0.0,NaN,NaN,Unknown,No,No,0.0,354,4.936458,0.425360,0,No,NaN,NaN,Unknown,0,0.0,0,2420,0.0,0,2255,1.0,1
7267,P2S1,2019-03-20,10,1,West,T1,H2,large,2.0,No,NaN,NaN,Unknown,0.0,0.0,0,0.0,NaN,NaN,Unknown,No,No,0.0,354,3.394271,1.412394,0,No,NaN,NaN,Unknown,0,0.0,0,2420,0.0,0,2255,1.0,1
7268,P2S1,2019-03-21,9,1,West,T1,H2,large,2.0,No,NaN,NaN,Unknown,0.0,0.0,0,0.0,NaN,NaN,Unknown,No,No,0.0,354,4.167917,3.042049,0,No,NaN,NaN,Unknown,0,0.0,0,2420,0.0,0,2255,1.0,1
7269,P2S1,2019-03-22,8,1,West,T1,H2,large,2.0,No,NaN,NaN,Unknown,0.0,0.0,0,0.0,NaN,NaN,Unknown,No,No,0.0,354,4.028187,0.000410,0,No,NaN,NaN,Unknown,0,0.0,0,2420,0.0,0,2255,1.0,1


## Set up for Model

In [ ]:
X = df.drop(columns=["duration", "event_observed", "start_date", "randid"])
y_time = df["duration"].values
y_event = df["event_observed"].values

X_encoded = pd.get_dummies(X, drop_first=True)

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_encoded)

X_train, X_test, y_train_time, y_test_time, y_train_event, y_test_event = train_test_split(
    X_scaled, y_time, y_event, test_size=0.2, random_state=42)

In [ ]:
X_train.shape, X_test.shape, y_train_time.shape, y_test_time.shape, y_train_event.shape, y_test_event.shape

((5816, 58), (1455, 58), (5816,), (1455,), (5816,), (1455,))

## Set up Model + Train

In [ ]:
labtrans = DeepHitSingle.label_transform(100)
y_train_dh = labtrans.fit_transform(y_train_time, y_train_event)
y_test_dh = labtrans.transform(y_test_time, y_test_event)

In [ ]:
# restructure before training
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')

In [ ]:
in_features = X_train.shape[1]
out_features = labtrans.out_features
net = tt.practical.MLPVanilla(in_features, [64, 64], out_features, batch_norm=True, dropout=0.1)

model = DeepHitSingle(net, tt.optim.Adam, alpha=0.2, sigma=0.1, duration_index=labtrans.cuts)
model.optimizer.set_lr(1e-3)

batch_size = 256
epochs = 100
callbacks = [tt.callbacks.EarlyStopping(patience=10)]

log = model.fit(X_train, y_train_dh, batch_size, epochs, callbacks, verbose=True, val_data=(X_test, y_test_dh))

0:	[0s / 0s],		train_loss: 1.0360,	val_loss: 0.9895
1:	[0s / 0s],		train_loss: 0.9627,	val_loss: 0.9247
2:	[0s / 0s],		train_loss: 0.9127,	val_loss: 0.8714
3:	[0s / 0s],		train_loss: 0.8643,	val_loss: 0.8242
4:	[0s / 0s],		train_loss: 0.8261,	val_loss: 0.7939
5:	[0s / 0s],		train_loss: 0.8026,	val_loss: 0.7734
6:	[0s / 1s],		train_loss: 0.7796,	val_loss: 0.7520
7:	[0s / 1s],		train_loss: 0.7659,	val_loss: 0.7363
8:	[0s / 1s],		train_loss: 0.7464,	val_loss: 0.7223
9:	[0s / 1s],		train_loss: 0.7408,	val_loss: 0.7187
10:	[0s / 1s],		train_loss: 0.7278,	val_loss: 0.7108
11:	[0s / 1s],		train_loss: 0.7187,	val_loss: 0.7072
12:	[0s / 1s],		train_loss: 0.7208,	val_loss: 0.6962
13:	[0s / 2s],		train_loss: 0.7059,	val_loss: 0.6912
14:	[0s / 2s],		train_loss: 0.7050,	val_loss: 0.6866
15:	[0s / 2s],		train_loss: 0.6961,	val_loss: 0.6826
16:	[0s / 2s],		train_loss: 0.6946,	val_loss: 0.6835
17:	[0s / 2s],		train_loss: 0.6920,	val_loss: 0.6773
18:	[0s / 2s],		train_loss: 0.6858,	val_loss: 0.6780
19:

## Evaluate

In [ ]:
surv = model.predict_surv_df(X_test)

ev = EvalSurv(surv, y_test_time, y_test_event, censor_surv='km')
print("DeepHit C-index:", round(ev.concordance_td(), 4))

DeepHit C-index: 0.6267
